# Network Benchmark Austria - Data Preparation

Prepares data to be used for Network Benchmark Austria Dash application. Source Data can be downloaded   
from [RTR Opendata Website](https://www.netztest.at/de/Opendata).



In [4]:
import pandas as pd
import zipfile
import matplotlib.pyplot as plt
import geopandas as gpd
import geoplot
from shapely.geometry import Point
from geopandas.tools import sjoin

#%matplotlib inline

In [5]:
pd.__version__

'0.23.4'

In [6]:
def read_csv(filename):
    return pd.read_csv(zipfile.ZipFile(filename + '.zip').open(filename + '.csv'))

def prepare_original_df(filelist):
    df = pd.DataFrame()
    for f in filelist:
        print('Reading {}'.format(f))
        df = df.append(read_csv(f))
    return df
        
def clean_data(df_original):
    #selecting only mobile network tests
    df_original.reset_index(inplace=True)
    mobile_only_tech = ['4G', '3G', 'MOBILE', '3G/4G', '2G', '2G/3G','2G/3G/4G']
    
    #selecting only tests conducted in austria, by selecting 'AT' from country location
    df_clean = df_original[(df_original.cat_technology.isin(mobile_only_tech)) & (df_original.country_location == 'AT')]
    
    #dropping columns without Network Code, Position Info
    #seemingly from platform ios network_mcc_mnc is not returned, using sim_mcc_mnc instead (even though slight discrepancies)
    df_clean.loc[df_clean['platform']=='iOS', 'network_mcc_mnc']=df_clean.sim_mcc_mnc

    
    dropcols = ['network_mcc_mnc', 'loc_accuracy']
    df_clean = df_clean.dropna(subset = dropcols)
    
    #selecting columns to be kept for further analysis
    keep_cols = ['open_uuid', 'open_test_uuid', 'time_utc', 'cat_technology',
       'network_type', 'lat', 'long', 'gkz',
       'zip_code', 'download_kbit', 'upload_kbit',
       'ping_ms','platform', 'model', 'client_version', 'network_mcc_mnc',
       'network_name', 'sim_mcc_mnc', 'cell_area_code', 'cell_location_id']
    df_clean = df_clean[keep_cols]

    return df_clean

def extract_time_features(df):
    print('Extrating Time Features')
    #df['time2'] = df.apply(lambda x: datetime.datetime.strptime(x[col], '%Y-%m-%d %H:%M:%S'), axis=1)
    #df['time2'] = df.apply(lambda x: x['time2'].to_datetime64(), axis=1)
    df['year'] = df.apply(lambda x: x['time'].year, axis=1)
    df['month'] = df.apply(lambda x: x['time'].month, axis=1)
    df['day'] = df.apply(lambda x: x['time'].day, axis=1)
    df['hour'] = df.apply(lambda x: x['time'].hour, axis=1)
    df['weekday'] = df.apply(lambda x: x['time'].isoweekday(), axis=1)
    #df.drop(labels = 'time2', inplace=True, axis=1)
    
    return df


def add_data(df_clean):
    
    #map mobile networks based on MNC (taken from wikipedia, and dropping those that can't be mapped
    print('Adding & Mapping Data')
    df_clean['network'] = df_clean.apply(lambda x: map_mobile_network(x['network_mcc_mnc']), axis=1)
    df_clean = df_clean.dropna(subset = ['network'])
    
    #converting string timestamp to pandas datetime
    df_clean['time'] = pd.to_datetime(df_clean['time_utc'])
    df_clean = df_clean.drop(labels = ['time_utc'], axis=1)
    df_clean['download_Mbit'] = df_clean['download_kbit']/1024
    df_clean['upload_Mbit'] = df_clean['upload_kbit']/1024
    df_clean = extract_time_features(df_clean)
    
    return df_clean





def map_mobile_network(mnc):
    #list of MNCs - taken from wikipedia
    network_operators = {'A1': {'232-01', '232-02', '232-09', '232-11', '232-12'},
                    'TMA': {'232-03', '232-04', '232-07'},
                    'H3A': {'232-05', '232-10', '232-14', '232-16'}}
    mapped = False
    for k in network_operators.keys():
        if mnc in network_operators[k]:
            net = k
            mapped = True
        
    if mapped == False:
        net = pd.np.nan
    return net

def map_to_districts(df):
    #load a shapefile with all districts in austria
    shapefile = 'http://data.statistik.gv.at/data/OGDEXT_POLBEZ_1_STATISTIK_AUSTRIA_20180101.zip'
    print('Loading Shapefile from {}'.format(shapefile))
    df_bez = gpd.read_file(shapefile)
    
    #reset crs to 4326 to plot easily
    crs = 4326
    print('Setting crs to {}'.format(crs))
    df_bez = df_bez.to_crs(epsg = crs)
    
    #converting dataframe to GeoDataframe
    print('Converting Dataframe to GeoDataframe')
    #make a tuple from lon, lat and transform it into Geometry Points Objects
    df['Coordinates'] = list(zip(df.long, df.lat))
    #transform to points
    df.Coordinates = df.Coordinates.apply(Point)
    #make Geodataframe out of it
    geo_df = gpd.GeoDataFrame(df, geometry = 'Coordinates')
    geo_df.crs = {'init' :'epsg:4326', 'no_defs':True}
    
    
    #map points in geodataframe to district shapefiles
    geo_df= sjoin(geo_df, df_bez, how='left')
    geo_df.rename(columns={'NAME': 'district'}, inplace=True)
    
    print('Done!')
    return geo_df

def save_final_df(df, filename):
    print('Saving file as {}'.format(filename))
    df.to_csv(filename) 


def prepare_data(filelist, filename):
    df = prepare_original_df(filelist)
    df_clean = clean_data(df)
    df_final = add_data(df_clean)
    df_mapped = map_to_districts(df_final)
    save_final_df(df_mapped, filename)
    print('Done!')
    return df_final

    
    
    

In [7]:
# Load RTR data, select mobile only, map to networks and store in a dataframe
filelist = ['netztest-opendata-2019-01','netztest-opendata-2018-12','netztest-opendata-2018-11',
            'netztest-opendata-2018-10','netztest-opendata-2018-09','netztest-opendata-2018-08',
            'netztest-opendata-2018-07','netztest-opendata-2018-06','netztest-opendata-2018-05',
            'netztest-opendata-2018-04','netztest-opendata-2018-03','netztest-opendata-2018-02',
            'netztest-opendata-2018-01']

df = prepare_data(filelist, 'rtr-data.csv' )

Reading netztest-opendata-2019-01
Reading netztest-opendata-2018-12
Reading netztest-opendata-2018-11
Reading netztest-opendata-2018-10
Reading netztest-opendata-2018-09
Reading netztest-opendata-2018-08
Reading netztest-opendata-2018-07
Reading netztest-opendata-2018-06
Reading netztest-opendata-2018-05
Reading netztest-opendata-2018-04
Reading netztest-opendata-2018-03
Reading netztest-opendata-2018-02
Reading netztest-opendata-2018-01


C:\Users\creze\Anaconda3\envs\py36\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Adding & Mapping Data


C:\Users\creze\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Extrating Time Features
Loading Shapefile from http://data.statistik.gv.at/data/OGDEXT_POLBEZ_1_STATISTIK_AUSTRIA_20180101.zip
Setting crs to 4326
Converting Dataframe to GeoDataframe
Done!
Saving file as rtr-data.csv
Done!


In [5]:
df_new = pd.read_csv('rtr-data.csv')
df_new.head()

,Unnamed: 0,open_uuid,open_test_uuid,cat_technology,network_type,lat,long,gkz,zip_code,download_kbit,...,upload_Mbit,year,month,day,hour,weekday,Coordinates,index_right,ID,district
0,10,Pe3506338-fb07-49ad-8391-619e909806aa,Oa57fc330-f601-4da6-9872-bdc26b79a741,4G,LTE,48.194569,16.260435,90001.0,NaN,48844,...,9.191406,2018,12,1,0,6,POINT (16.26043475234532 48.19456852045674),93.0,900.0,Wien(Stadt)
1,60,P1ad74b50-eb6a-4f6e-bf47-be16f78e7030,Of979dd65-4672-4f32-9b0f-3f0a08073510,4G,LTE,48.528415,13.760682,41329.0,NaN,24793,...,5.916992,2018,12,1,0,6,POINT (13.7606815 48.528415),55.0,413.0,Rohrbach
2,70,Pb35b0f06-6587-474a-8f89-33be166dabba,O697ebb09-5416-4dc7-bc68-ebbb94dc0969,4G,LTE,48.329091,14.979789,31506.0,NaN,53835,...,12.507812,2018,12,1,0,6,POINT (14.9797885 48.3290912),33.0,315.0,Melk
3,135,Pa8a2b33f-113a-437d-a2b9-7774339cc537,Oe0263457-b8d4-4846-8cc2-f7b6a413ee69,4G,LTE,48.152141,16.311808,90001.0,NaN,34288,...,4.630859,2018,12,1,0,6,POINT (16.3118082 48.1521406),93.0,900.0,Wien(Stadt)
4,148,P24211109-a557-438c-b5b9-deeed4aad73f,O7cf7f8eb-820a-489a-acb5-e8089a91fb6a,4G,LTE,48.843564,15.087130,30902.0,NaN,60640,...,27.843750,2018,12,1,1,6,POINT (15.0871302180637 48.84356400816684),27.0,309.0,Gmünd
